# Non manhattan routing

gdsfactory provides functions to connect and route components ports that are off-grid or have non manhattan orientations (0, 90, 180, 270 degrees)

## Fix Non manhattan connections

In [ ]:
import gdsfactory as gf
from gdsfactory.decorators import has_valid_transformations


@gf.cell
def demo_non_manhattan():
    c = gf.Component("bend")
    b = c << gf.components.bend_circular(angle=30)
    s = c << gf.components.straight(length=5)
    s.connect("o1", b.ports["o2"])
    return c


c1 = demo_non_manhattan()
print(has_valid_transformations(c1))
c1

if you zoom in between the bends you will see a notch between waveguides due to non-manhattan connection between the bends.

![gap](https://i.imgur.com/jBEwy9T.png)

You an fix it with the `flatten_invalid_refs` flag when you call `Component.write_gds()`.

In [ ]:
help(c1.write_gds)

In [ ]:
gdspath = c1.write_gds(flatten_invalid_refs=True)
c2 = gf.import_gds(gdspath)
c2

In [ ]:
has_valid_transformations(c1)  # has gap issues

In [ ]:
has_valid_transformations(c2)  # works perfect

If you zoom in the connection the decorator you can see it fixed the issue in `c` that we fixed in `c2` thanks to the `flatten_invalid_refs` flag.

![no gap](https://i.imgur.com/VbSgIjP.png)

## Non manhattan router

In [ ]:
from gdsfactory.cell import cell
from gdsfactory.component import Component
from gdsfactory.read import from_yaml


@cell
def demo_all_angle_routing() -> Component:
    """Demonstrate all-angle routing."""
    yaml = """
    instances:
        mmi_long:
          component: mmi1x2
          settings:
            width_mmi: 4.5
            length_mmi: 10
        mmi_short:
          component: mmi1x2
          settings:
            width_mmi: 4.5
            length_mmi: 5

    placements:
        mmi_long:
            rotation: 190
            x: 100
            y: 100

    routes:
        optical:
            routing_strategy: get_bundle_all_angle
            settings:
                steps:
                    - ds: 50
                      exit_angle: 90  # TODO: why do paths cross when set to i.e. 100?
            links:
                mmi_short,o2: mmi_long,o3
                mmi_short,o3: mmi_long,o2

    ports:
        o2: mmi_short,o1
        o1: mmi_long,o1
    """

    return from_yaml(yaml)


c = demo_all_angle_routing()
c

In [ ]:
import gdsfactory as gf

c = gf.Component("demo")

mmi = gf.components.mmi2x2(width_mmi=10, gap_mmi=3)
mmi1 = c << mmi
mmi2 = c << mmi

mmi2.move((100, 10))
mmi2.rotate(30)

routes = gf.routing.get_bundle_all_angle(
    mmi1.get_ports_list(orientation=0),
    [mmi2.ports["o2"], mmi2.ports["o1"]],
    connector=None,
)
for route in routes:
    c.add(route.references)
c

In [ ]:
c = gf.Component("demo")

mmi = gf.components.mmi2x2(width_mmi=10, gap_mmi=3)
mmi1 = c << mmi
mmi2 = c << mmi

mmi2.move((100, 10))
mmi2.rotate(30)

routes = gf.routing.get_bundle_all_angle(
    mmi1.get_ports_list(orientation=0),
    [mmi2.ports["o2"], mmi2.ports["o1"]],
    connector='low_loss',
)
for route in routes:
    c.add(route.references)
c

In [ ]:
import gdsfactory as gf
from gdsfactory.routing.all_angle import get_bundle_all_angle

NUM_WIRES=10

@gf.cell
def inner_array():
    c = gf.Component()
    base = gf.components.straight(cross_section=gf.cross_section.strip).rotate(45)
    for x in range(10):
        for y in range(6):
            base_ref = c.add_ref(base).move((x*20 - 90, y*20 - 50))
            c.add_port(f"inner_{x}_{y}", port=base_ref.ports['o1'])
    return c

@gf.cell
def outer_array():
    c = gf.Component()
    base = gf.components.straight(cross_section=gf.cross_section.strip)
    for idx, theta in enumerate(range(0, 360, 6)):
        base_ref = c.add_ref(base).move((300, 0)).rotate(theta)
        c.add_port(f"outer_{idx}", port=base_ref.ports['o1'])
    return c

@gf.cell
def chip():
    c = gf.Component()
    inner = c << inner_array()
    outer = c << outer_array()
    inner_ports = inner.get_ports_list()
    outer_ports = outer.get_ports_list()
    for n_route in range(NUM_WIRES):
        routes = get_bundle_all_angle(
            ports1=[inner_ports[n_route]],
            ports2=[outer_ports[n_route]],
            cross_section=gf.cross_section.strip,
            bend=gf.components.bend_euler,
            start_angle=-40,
            steps=[{'ds': (NUM_WIRES - n_route) * 20},]
        )
        for route in routes:
            c.add(route.references)
    return c

gf.get_active_pdk().register_cross_sections(strip=gf.cross_section.strip)
c = chip()
c